# Exercicio PySpark

Exercicio proposto para a aula de Storage.

In [1]:
!pip install pyspark findspark

In [10]:
!hdfs dfsadmin -safemode leave
!hdfs dfs -mkdir /user/root/
!hdfs dfs -put /mnt/notebooks/src/titanic-dataset.csv /user/root/titanic-dataset.csv

Safe mode is OFF


In [38]:
import findspark
findspark.init()
findspark.find()

'/opt/spark'

In [49]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()

In [50]:
from pprint import pprint

In [51]:
df = spark.read.format("csv").option("header","true").load("titanic-dataset.csv")

### Renomeando colunas para ficar mais amigável

In [57]:
ports = {
    "C": "Cherbourg",
    "Q": "Queenstown",
    "S": "Southampton",
}

In [55]:
df = df.withColumnRenamed(
    "PassengerId", "passenger_id"
).withColumnRenamed(
    "Survived", "survived"
).withColumnRenamed(
    "Pclass", "passenger_class"
).withColumnRenamed(
    "Sex", "sex"
).withColumnRenamed(
    "Age", "age"
).withColumnRenamed(
    "SibSp", "siblings_and_spouses"
).withColumnRenamed(
    "Parch", "number_of_parents_or_children_onboard"
).withColumnRenamed(
    "Ticket", "ticket"
).withColumnRenamed(
    "Fare", "fare"
).withColumnRenamed(
    "Cabin", "cabin"
).withColumnRenamed(
    "Embarked", "port_of_origin"
).withColumn(
    "port_of_origin", port_renamer(col("port_of_origin"))
)

### Analise do dataset

In [56]:
df.limit(5).show() # Preview do dataset

Py4JJavaError: An error occurred while calling o265.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 69.0 failed 1 times, most recent failure: Lost task 0.0 in stage 69.0 (TID 1215, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 267, in main
    ("%d.%d" % sys.version_info[:2], version))
Exception: Python in worker has different version 3.5 than that in driver 3.6, PySpark cannot run with different minor versions.Please check environment variables PYSPARK_PYTHON and PYSPARK_DRIVER_PYTHON are correctly set.

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:452)
	at org.apache.spark.sql.execution.python.PythonUDFRunner$$anon$1.read(PythonUDFRunner.scala:81)
	at org.apache.spark.sql.execution.python.PythonUDFRunner$$anon$1.read(PythonUDFRunner.scala:64)
	at or

In [44]:
df.schema # Schema do modelo

StructType(List(StructField(passenger_id,StringType,true),StructField(survived,StringType,true),StructField(passenger_class,StringType,true),StructField(Name,StringType,true),StructField(sex,StringType,true),StructField(age,StringType,true),StructField(siblings_and_spouses,StringType,true),StructField(number_of_parents_or_children_onboard,StringType,true),StructField(ticket,StringType,true),StructField(fare,StringType,true),StructField(cabin,StringType,true),StructField(port_of_origin,StringType,true)))

In [45]:
df.groupBy(col("survived")).count().show() # Quantidade de pessoas que sobreviveram ao Titanic

+--------+-----+
|survived|count|
+--------+-----+
|       0|  549|
|       1|  342|
+--------+-----+



In [46]:
df.groupBy(col("survived"), col("sex")).count().show() # Quantidade de pessoas que sobreviveram ao Titanic

+--------+------+-----+
|survived|   sex|count|
+--------+------+-----+
|       1|  male|  109|
|       1|female|  233|
|       0|female|   81|
|       0|  male|  468|
+--------+------+-----+



In [47]:
df.groupBy(col("survived"), col("sex"), col("passenger_class")).count().show()

+--------+------+---------------+-----+
|survived|   sex|passenger_class|count|
+--------+------+---------------+-----+
|       0|female|              1|    3|
|       0|female|              2|    6|
|       1|female|              3|   72|
|       1|  male|              1|   45|
|       1|  male|              3|   47|
|       1|female|              2|   70|
|       0|  male|              3|  300|
|       0|female|              3|   72|
|       0|  male|              2|   91|
|       1|female|              1|   91|
|       0|  male|              1|   77|
|       1|  male|              2|   17|
+--------+------+---------------+-----+

